<h1 align="center">
Too Long; Didn't Read ¯\_(ツ)_/¯
</h1>

This interactive notebook uses 🤗 Hugging Face Transformers & BART to generate summaries from inputted text. To live up to this notebook's theme (tl;dr) I will end the description there.

In [ ]:
# Import necessary libraries
import ipywidgets as widgets
from IPython.display import display, clear_output
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Suppressing any warnings for this notebook but generally not needed or advised
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [ ]:
# Load BART model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-cnn')
model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large-cnn')

In [ ]:
MINIMUM_INPUT_CHARS = 500

# Extract text from text box
def extract_text(text):
    return text

# Generate summary after tokenizing text 
def generate_summary(content):
    inputs = tokenizer.encode(
        content,
        truncation=True,
        return_tensors='pt',
        max_length=tokenizer.model_max_length,
    )

    summary_ids = model.generate(
        inputs,
        num_beams=5,
        max_length=100,
        length_penalty=2.0,
        early_stopping=True
    )

    summary = tokenizer.decode(
        summary_ids[0],
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )
    return summary

# Clear the output when changes occur in the input text box and display
# a loading spinner while the text is being analyzed
def update_output(content):
    with output:
        clear_output(wait=True)
        content = extract_text(text_box.value)

        if content and len(content) >= MINIMUM_INPUT_CHARS:
            loading_spinner = widgets.HTML(
                f'<div style="display: flex; justify-content: center;"> \
                  <i class="fa fa-spinner fa-pulse fa-3x fa-fw"></i></div>'
            )
            display(loading_spinner)
            summary = generate_summary(content)
            clear_output(wait=True)
            print('Summary:\n\n', summary)

        elif content:
            clear_output(wait=True)
            print('The text entered is too short for effective summarization. '
                  'Please enter text that is at least 500 characters or more in length.')

# Use to clear summary output box when the cell is loaded/re-loaded
def clear_output_summary():
    with output:
        clear_output(wait=False)

# Create text input box
text_box = widgets.Textarea(
    placeholder='Enter or paste text here...',
    layout={'height': '500px',
            'width': '750px'}
    )

# Create output box to show the summary
output = widgets.Output(
    layout={'border': '2px dashed black',
            'height': '500px',
            'width': '750px'}
    )

# Observer updates the output when a change occurs in input text
text_box.observe(update_output, names='value')

# Horizontal side-by-side ouput cell layout
cell_layout = widgets.HBox([text_box, output])

display(cell_layout)
clear_output_summary()